In [1]:
from DCS import *
from sentences import *
import os
from utilities import *
import re
from romtoslp import rom_slp
import pickle

In [2]:
dcsPath = '../Text Segmentation/DCS_pick/'
dcsList = os.listdir(dcsPath)

In [3]:
sktPath_1 = '../TextSegmentation/corrected_10to20/'
sktList_1 = os.listdir(sktPath_1)

In [4]:
def pickleFixLoad(filename):
    return pickle.load(open(filename, 'rb'), encoding=u'utf8')

In [7]:



def CanIUseIt(sntcObj, dcsObj):
#     print('-'*15)
#     CHECK 1 -> NUMBER OF CHUNKS/LEMMAS
    s = re.findall("[^ ]+", dcsObj.sentence)
    if((len(s) != len(dcsObj.lemmas)) or (len(s) != len(sntcObj.chunk))):
        return 1
    
#     CHECK 2 -> ALL LEMMAS PRESENT IN CORRESPONDING CHUNK
    sntc_lemma_packs = []
    for chunk in sntcObj.chunk:
        allwords = []
        for pos, word_senses in chunk.chunk_words.items():
            for sense in word_senses:
                for lemma in sense.lemmas:
                    allwords.append(rom_slp(lemma.split('_')[0]))
        sntc_lemma_packs.append(allwords)
        
    i = -1
    for chunk in dcsObj.lemmas:
        chunk = list(map(rom_slp, chunk))
        i += 1
#         print(" | ".join(chunk))
#         print("vs")
#         print(" | ".join(sntc_lemma_packs[i]))
#         print()
        for lemma in chunk:
            if(lemma not in sntc_lemma_packs[i]):
#                 print(dcsObj.sentence)
                return 2
    
#     CHECK 3 -> NO CNG ERRORS ('VERBFORMS')


common = []
bads = []
for dcsFile in dcsList[0:100]:
    if(validatePickleName(dcsFile) != ""):
        if dcsFile in sktList_1:
            common.append(dcsFile)            
            dcsObj = pickleFixLoad(dcsPath + dcsFile)
            sntcObj = pickleFixLoad(sktPath_1 + dcsFile)
            e = CanIUseIt(sntcObj, dcsObj)
            # IF zero returned then all test passed
            if e != 0:
                bads.append(dcsFile)
            if e == 2:
#                 print(dcsFile)
                pass
#                 print("FAIL 2")

print(len(common))
print(len(bads))

41
41


In [25]:
len(common)

41

In [26]:
len(bads)

41